<a href="https://colab.research.google.com/github/tonytarizzo/paddyRiceCompetition/blob/main/paddyCompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai
!pip install timm

try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *
from fastai.vision.all import*
import timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
set_seed(42)
comp = 'paddy-disease-classification'

path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')
path.ls()

100%|██████████| 1.02G/1.02G [00:50<00:00, 21.5MB/s]


(#4) [Path('paddy-disease-classification/train_images'),Path('paddy-disease-classification/train.csv'),Path('paddy-disease-classification/sample_submission.csv'),Path('paddy-disease-classification/test_images')]

In [5]:
#Rotate all images to correct orientation
trn_path = path/'train_images'
files = get_image_files(trn_path)

from fastcore.parallel import *
from PIL import Image as PILImage

def f(o):
    return (o, PILImage.open(o).size)

def rotate_and_save(filepath):
    with PILImage.open(filepath) as img:
        rotated = img.rotate(90, expand=True)
        rotated.save(filepath)

def fix_rotation(files):
    sizes = parallel(f, files, n_workers=8)
    incorrect_size_files = [f[0] for f in sizes if f[1] != (480, 640)]
    parallel(rotate_and_save, incorrect_size_files, n_workers=8)
    return files

files = fix_rotation(files)

tst_path = path/'test_images'
tst_files = sorted(get_image_files(tst_path))
tst_files = fix_rotation(tst_files)


In [6]:
def train(arch, size, item=Resize(480, method='squish'), epochs=12, accum=4, finetune=True, lr=0.01):
    dls = ImageDataLoaders.from_folder(trn_path, seed=42, valid_pct=0.2, item_tfms=item, batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    #slide_lr = learn.lr_find(suggest_funcs=(slide), show_plot=False)
    if finetune:
        learn.fine_tune(epochs, lr)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, lr)
        #return slide_lr

In [7]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [8]:
#Testing GPU Load
!pip install pynvml
#trn_path = path/'train_images'/'bacterial_panicle_blight'

#train('convnext_large_in22k', size=(320,240), epochs=1, accum=2, finetune=False)
#report_gpu()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
#Training One Model Only
trn_path = path/'train_images'

learn,slide_lr = train('convnext_large_in22k', size=(320,240), epochs=12, accum=2)

In [ ]:
#Saving results/submission from One Model Only
idxs = tta_probs.argmax(dim=1)
vocab = np.array(learn.dls.vocab)
results = pd.Series(vocab[idxs], name="idxs")

ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = results
ss.to_csv('subm.csv', index=False)
!head subm.csv

In [9]:
#Training Model Ensemble & Saving .pkl
#trn_path = path/'train_images'
res = 640,480

models = {
    'convnext_large.fb_in22k': {(Resize(res), (320,224)),
                                (Resize(res), 192),},
    'vit_large_patch16_224': {(Resize(480, method='squish'), 224),
                              (Resize(res), 224),
                              (Resize(res, method='squish'), 224)},
    'beit_base_patch16_384': {(Resize(res), 384),}
}

tta_res = []

for arch, details in models.items():
    for item, size in details:
        print('ARCH: ', arch)
        print('SIZE: ', size)
        print("ITEM: ", item)
        tta_res.append(train(arch, size, item=item, accum=2, epochs=9, finetune=True,lr=0.006))
        gc.collect()
        torch.cuda.empty_cache()

save_pickle('tta_res.pkl', tta_res)

ARCH:  convnext_large.fb_in22k
SIZE:  (320, 224)
ITEM:  Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}:
encodes: (Image,object) -> encodes
(TensorBBox,object) -> encodes
(TensorPoint,object) -> encodes
decodes: 


epoch,train_loss,valid_loss,error_rate,time
0,0.839759,0.491172,0.165305,00:48


epoch,train_loss,valid_loss,error_rate,time
0,0.418320,0.253358,0.076886,00:57
1,0.325318,0.234814,0.070639,00:56
2,0.280803,0.227025,0.069678,00:56
3,0.172524,0.141043,0.043729,00:56
4,0.126341,0.167655,0.041807,00:56
5,0.096037,0.112679,0.030754,00:56
6,0.071303,0.103322,0.027391,00:56
7,0.051208,0.097634,0.025949,00:56
8,0.032190,0.094205,0.023546,00:56


ARCH:  convnext_large.fb_in22k
SIZE:  192
ITEM:  Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}:
encodes: (Image,object) -> encodes
(TensorBBox,object) -> encodes
(TensorPoint,object) -> encodes
decodes: 


epoch,train_loss,valid_loss,error_rate,time
0,0.849474,0.483841,0.153772,00:30


...

epoch,train_loss,valid_loss,error_rate,time
0,0.408283,0.252608,0.084094,00:46
1,0.364515,0.265264,0.078808,00:46
2,0.272650,0.261768,0.079289,00:46
3,0.234704,0.257832,0.078808,00:46
4,0.148287,0.171018,0.048054,00:46
5,0.097575,0.112401,0.031235,00:46
6,0.053562,0.112368,0.029313,00:46
7,0.045142,0.097984,0.024988,00:46
8,0.033401,0.096465,0.026430,00:46


ARCH:  beit_base_patch16_384
SIZE:  384
ITEM:  Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}:
encodes: (Image,object) -> encodes
(TensorBBox,object) -> encodes
(TensorPoint,object) -> encodes
decodes: 


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


epoch,train_loss,valid_loss,error_rate,time
0,0.851645,0.529465,0.160019,00:55


epoch,train_loss,valid_loss,error_rate,time
0,0.415265,0.215980,0.062470,01:07
1,0.357949,0.247528,0.075444,01:07
2,0.273291,0.156235,0.048054,01:07
3,0.254704,0.146734,0.047093,01:07
4,0.141740,0.146151,0.043248,01:07
5,0.124416,0.103057,0.031235,01:07
6,0.073941,0.091346,0.023066,01:07
7,0.049494,0.070043,0.021624,01:07
8,0.048273,0.067917,0.022105,01:07


In [10]:
#Saving results/submission from Model Ensemble
tta_prs = first(zip(*tta_res))
tta_prs += tta_prs[1:3]

avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = vocab[idxs]
ss.to_csv('subm.csv', index=False)
!head subm.csv

image_id,label
200001.jpg,hispa
200002.jpg,normal
200003.jpg,blast
200004.jpg,blast
200005.jpg,blast
200006.jpg,brown_spot
200007.jpg,dead_heart
200008.jpg,brown_spot
200009.jpg,hispa


In [11]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'my comment', comp)

100%|██████████| 70.5k/70.5k [00:02<00:00, 30.6kB/s]
